In [320]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import periodogram, welch
from scipy import signal
import seaborn as sns
import ipywidgets as widgets
from IPython.display import clear_output
from ipywidgets import interact, FloatSlider, IntSlider

In [321]:
fs = 1000
t = np.linspace(0, 1, fs) 
f = 20

In [322]:
def comapre_signals(sygnal1, sygnal2_1, sygnal2_2 = None, xmin = 0, xmax = 1, name = None, original = True):
    
    sns.set()

    fig, axs = plt.subplots(1, 2, figsize=(16, 4))

    axs[0].plot(sygnal1[0],sygnal1[1])
    if original:
        axs[0].set_title(f'Oryginalny sygnał {name}')
        axs[0].set_xlabel('Czas')
        axs[0].set_ylabel('Amplituda')
    else:
        axs[0].set_title(f'Widmowa gęstość mocy {name}')
        axs[0].set_xlabel('Częstotliwość')
        axs[0].set_ylabel('Gęstość mocy')
    axs[0].set_xlim(xmin,xmax)

    sns.lineplot(x=sygnal2_1[0], y=sygnal2_1[1], label='Periodogram', ax=axs[1])
    if sygnal2_2: sns.lineplot(x=sygnal2_2[0], y=sygnal2_2[1], label='Welch', ax=axs[1])
    axs[1].set_title(f'Widmowa gęstość mocy {name}')
    axs[1].set_xlabel('Częstotliwość')
    axs[1].set_ylabel('Gęstość mocy')
    axs[1].legend()
    axs[1].set_xlim(xmin,100)
    

    plt.tight_layout()
    plt.show()
      
def wgm(sygnal):
    
    f_periodogram, Pxx_periodogram = periodogram(sygnal, fs=fs)

    f_welch, Pxx_welch = welch(sygnal, fs=fs)

    return [f_periodogram, Pxx_periodogram], [f_welch, Pxx_welch]

def handmade_wgm(signal):
        N = len(signal)
        dt = 1 / fs
        Tau = np.arange(-N+1, N) * dt # Tworzymy Tau osobno na podstawie częstotliwości próbkowania
        freqs = np.linspace(-fs / 2, fs / 2, len(signal))
    

        R_xx = np.correlate(signal, signal, mode='full') / N
        S_xx = []

        # Całka w formie listy sum
        for f in freqs:
            exp_term = np.exp(-1j * 2 * np.pi * f * Tau)  # e^(-i2πfτ)
            integral = np.sum(R_xx * exp_term) * dt  
            # sumowanie ręczne zamiast całki, bo funkcja całki nie pozwala nam wprowadzic zmiennych w tej formie
            S_xx.append(np.abs(integral))


        return [freqs, S_xx],None

def make_signals():
    
    x_sin = np.sin(2 * np.pi * f * t)
    x_rect = signal.square(2 * np.pi * f * t)
    x_saw = signal.sawtooth(2 * np.pi * f * t)
    f1 = f
    f2 = 4*f
    x_chirp = signal.chirp(t, f0=f1, f1=f2, t1=t[-1], method='linear')
    x_super = np.sin(2 * np.pi * f1 * t) + 0.5 * np.cos(2 * np.pi * f2 * t)
    x_imp = np.zeros(len(t))
    x_imp[int(len(t)/2)] = 1    

    signals = {
    "sinusoidalny": x_sin,
    "prostokątny": x_rect,
    "piłokształtny": x_saw,
    "świergotliwy": x_chirp,
    "superpozycji": x_super,
    "impuls jednostkowy": x_imp
}
    return signals

In [323]:
signals = make_signals()

In [324]:
def show_signal_comparison(func1, func2 = False, overlapping = False, original = False):

    

    def interactive_plot(signal_name):
        clear_output(wait=True)
        s = signals[signal_name]
        s1, s2 = func1(s)
        if func2: s, s22 = func2(s)
        
        if overlapping and original:
            comapre_signals([t, s], s1, s2, name=signal_name, original=original)
        elif overlapping:
            comapre_signals(s, s1, s2, xmax = 100,name=signal_name, original=original)
        elif original:
            comapre_signals([t, s], s1, name=signal_name, original=original)
        else:
            print("Źle zrobiony wykres")

    signal_selector = widgets.Dropdown(
        options=list(signals.keys()),
        description='Sygnał:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    widgets.interact(interactive_plot, signal_name=signal_selector)


# Zadanie 1

In [ ]:
show_signal_comparison(wgm,overlapping = True, original = True)


interactive(children=(Dropdown(description='Sygnał:', layout=Layout(width='50%'), options=('sinusoidalny', 'pr…

# Zadanie 2

In [ ]:
show_signal_comparison(handmade_wgm,overlapping = False, original = True)

interactive(children=(Dropdown(description='Sygnał:', layout=Layout(width='50%'), options=('sinusoidalny', 'pr…

# Zadanie 3

In [ ]:
show_signal_comparison(wgm,handmade_wgm,overlapping = True, original = False)

interactive(children=(Dropdown(description='Sygnał:', layout=Layout(width='50%'), options=('sinusoidalny', 'pr…